In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir('gdrive/My Drive/crypto/')
#!unzip \*.zip


In [0]:
os.chdir('crypto_data')

In [4]:
import pandas as pd 
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , Lambda, Flatten, LSTM, CuDNNLSTM
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.optimizers import Adam ,RMSprop
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.utils.np_utils import to_categorical


from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization , MaxPooling2D, Activation

Using TensorFlow backend.


In [5]:
types = ['BTC-USD','LTC-USD','ETH-USD','BCH-USD']
full_df = pd.DataFrame()
for curr in types:
  df = pd.read_csv(f"{curr}.csv",names = ["time",'low','high','open',f'{curr}_close',f"{curr}_volume",])
  df.set_index("time",inplace = True)
  df = df[[f"{curr}_close",f"{curr}_volume"]]
  if len(full_df) == 0:
    full_df = df
  else:
    full_df = full_df.join(df)
print(full_df.head())



            BTC-USD_close  BTC-USD_volume  ...  BCH-USD_close  BCH-USD_volume
time                                       ...                               
1528968660    6489.549805        0.587100  ...     871.719971        5.675361
1528968720    6487.379883        7.706374  ...     870.859985       26.856577
1528968780    6479.410156        3.088252  ...     870.099976        1.124300
1528968840    6479.410156        1.404100  ...     870.789978        1.749862
1528968900    6479.979980        0.753000  ...     870.000000        1.680500

[5 rows x 8 columns]


In [7]:
CURR = "BTC-USD" #change this to train for different currency
SEQ = 60
FUTURE = 3
full_df[f'{CURR}_future'] = full_df[f'{CURR}_close'].shift(-FUTURE)
print(full_df.head())


            BTC-USD_close  BTC-USD_volume  ...  BCH-USD_volume  BTC-USD_future
time                                       ...                                
1528968660    6489.549805        0.587100  ...        5.675361     6479.410156
1528968720    6487.379883        7.706374  ...       26.856577     6479.979980
1528968780    6479.410156        3.088252  ...        1.124300     6480.000000
1528968840    6479.410156        1.404100  ...        1.749862     6477.220215
1528968900    6479.979980        0.753000  ...        1.680500     6480.000000

[5 rows x 9 columns]


In [0]:
def classify(curr,nex):
  if(float(curr)<float(nex)):
    return 1
  else:
    return 0 

In [8]:
full_df["profit"] = list(map(classify,full_df[f'{CURR}_close'],full_df[f'{CURR}_future']))
print(full_df.head(15))

            BTC-USD_close  BTC-USD_volume  ...  BTC-USD_future  profit
time                                       ...                        
1528968660    6489.549805        0.587100  ...     6479.410156       0
1528968720    6487.379883        7.706374  ...     6479.979980       0
1528968780    6479.410156        3.088252  ...     6480.000000       1
1528968840    6479.410156        1.404100  ...     6477.220215       0
1528968900    6479.979980        0.753000  ...     6480.000000       1
1528968960    6480.000000        1.490900  ...     6479.990234       0
1528969020    6477.220215        2.731950  ...     6478.660156       1
1528969080    6480.000000        2.174240  ...     6478.660156       0
1528969140    6479.990234        0.903100  ...     6479.339844       0
1528969200    6478.660156        3.258786  ...     6479.350098       1
1528969260    6478.660156        1.970352  ...     6479.990234       1
1528969320    6479.339844        3.056897  ...     6481.890137       1
152896

In [9]:
full_df.drop(columns= f'{CURR}_future', inplace= True)
for col in [cols for cols in full_df.columns if cols != 'profit']:
  full_df[col] = full_df[col].pct_change()
full_df.head()



,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume,profit
time,,,,,,,,,
1528968660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1528968720,-0.000334,12.126169,0.000828,31.588422,NaN,NaN,-0.000987,3.732135,0
1528968780,-0.001228,-0.599260,-0.000931,-0.754666,-0.000021,-0.675261,-0.000873,-0.958137,1
1528968840,0.000000,-0.545342,-0.000725,-0.906443,-0.000514,2.194860,0.000793,0.556401,0
1528968900,0.000088,-0.463713,-0.001140,71.690564,0.000515,1.865596,-0.000907,-0.039639,1


In [10]:
full_df.dropna(inplace=True)
full_df.head()


,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume,profit
time,,,,,,,,,
1528968780,-0.001228,-0.599260,-0.000931,-0.754666,-0.000021,-0.675261,-0.000873,-0.958137,1
1528968840,0.000000,-0.545342,-0.000725,-0.906443,-0.000514,2.194860,0.000793,0.556401,0
1528968900,0.000088,-0.463713,-0.001140,71.690564,0.000515,1.865596,-0.000907,-0.039639,1
1528968960,0.000003,0.979947,0.001349,-0.967606,0.000000,-0.903003,-0.000012,-0.006835,0
1528969020,-0.000429,0.832417,-0.000829,4.621710,-0.000021,10.445264,-0.000621,-0.481610,1


In [0]:
from collections import deque
abc = full_df
sequential_data = []
sequence = deque(maxlen = SEQ)
for dat in abc.values:
  sequence.append(dat[:-1])
  if len(sequence) == SEQ:
    sequential_data.append([np.array(sequence),dat[-1]])



  


In [13]:
ones = 0
zeroes = 0
for abc, res in sequential_data:
  if res == 0:
    zeroes = zeroes + 1
  else:
    ones = ones + 1 
print(zeroes/(zeroes+ones))
print(ones/(zeroes+ones))

0.5510479915628231
0.4489520084371768


In [0]:
import random
random.shuffle(sequential_data)

In [0]:
x = []
y = []
for xval , yval in sequential_data:
  x.append(xval)
  y.append(yval)
x = np.array(x)   


In [16]:
x.shape

(97663, 60, 8)

In [0]:
def standardise(j):
   return (j-xtrain.mean())/xtrain.std()
    

In [0]:
model = Sequential([Lambda(standardise, input_shape = (x.shape[1:])),
                    CuDNNLSTM(32,  return_sequences = True),
                    Dropout(0.2),
                    BatchNormalization(),
                    CuDNNLSTM(32, return_sequences = True),
                    Dropout(0.2),
                    BatchNormalization(),
                    CuDNNLSTM(32, return_sequences = False),
                    Dropout(0.2),
                    BatchNormalization(),
                    Dense(64, activation= 'relu'),
                    Dropout(0.2),
                    BatchNormalization(),
                    Dense(1,activation='sigmoid')
])
model.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.optimizer.lr=0.05

In [0]:
from sklearn.model_selection import train_test_split

xtrain,xval, ytrain, yval = train_test_split(x, y,test_size = 0.10, random_state = 42)
# batches=gen.flow(xtrain,ytrain,batch_size = 16)
# valbatches = gen.flow(xtest,ytest,batch_size = 16)

In [19]:
xtrain.shape

(87896, 60, 8)

In [0]:
history = model.fit(xtrain,ytrain,batch_size=64, epochs=10,callbacks=[EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')],validation_data=(xval,yval) )